<a href="https://colab.research.google.com/github/adriel312/ETL_Python_SDW2023/blob/main/ETL_python_Adriel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Essa é uma tabela com informações sobre o Campeonato Brasileiro de Futebol. Nela contem dados de todas as partidas entre os anos de 2003 a 2022. Algumas informações estão incompletas, pois não havia informação sobre dados dessa partida. Eu me propus a preencher os valores faltantes nas colucas de 'tecnico_mandante' e 'tecnico_visitante', para isso solicitei a API da OpenAI, as informações faltantes, e assim preenchi os dados NaN. Com isso implementei uma ETL utilizando Python, pandas e a API da OpenAI.

In [ ]:
# Link da base de dados em .csv: https://www.kaggle.com/datasets/adaoduque/campeonato-brasileiro-de-futebol/

## **E**xtract

Extrai todas as informações da partida, contidas no CSV. Após, faço um filtro para selecionar as partidas superiores a 2008, pois dados antes de 2009 não se mostraram precisos.

In [ ]:
import pandas as pd

df = pd.read_csv('campeonato-brasileiro-full.csv')
#user_ids = df['tecnico_mandante'].tolist()

#Converte a coluna 'data' para o tipo datetime com pandas
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

#Cria uma condição para selecionar as linhas com data superior a 2008
condicao = df['data'].dt.year > 2008

#Usa a condição e seleciona as linhas que atendem
df_filtrado = df[condicao]


print (df_filtrado)

## **T**ransform

Utilizei a API do OpenAI para buscar os nomes dos tecnicos faltantes na planilha CSV.

In [ ]:
# Redefine o índice com o método reset_index
df_filtrado.reset_index(drop=True, inplace=True)

print(df_filtrado.loc[0]['data'])

In [ ]:
!pip install openai

In [ ]:
import openai

# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai.api_key = 'TODO'

In [ ]:
#Teste para gerar o nome de apenas um técnico, ao invés de percorrer todo o dataframe
import openai

prompt = f"O nome do técnico do {df_filtrado.loc[0]['mandante']} na data de {df_filtrado.loc[0]['data']}, é:"
resposta = openai.Completion.create(
            engine="text-davinci-002",
            prompt=prompt,
            max_tokens=50,
            temperature=0.7,
)
tecnico_mandante = resposta.choices[0].text.strip()
df_filtrado.at[6 ,'tecnico_mandante'] = tecnico_mandante
print(df_filtrado.loc[6])

In [ ]:
# Função para preencher os nomes dos técnicos usando a API da OpenAI.
def preencher_tecnicos(row):
    if pd.isna(row['tecnico_mandante']):
        # Faça a chamada à API para preencher o técnico do time mandante.
        prompt = f"O técnico do time mandante {row['mandante']} em {row['data']} é: "
        resposta = openai.Completion.create(
            engine="text-davinci-002",
            prompt=prompt,
            max_tokens=50,
            temperature=0.7,
        )
        tecnico_mandante = resposta.choices[0].text.strip()
    else:
        tecnico_mandante = row['tecnico_mandante']

    if pd.isna(row['tecnico_visitante']):
        # Faça a chamada à API para preencher o técnico do time visitante.
        prompt = f"O técnico do time visitante {row['visitante']} em {row['data']} é: "
        resposta = openai.Completion.create(
            engine="text-davinci-002",
            prompt=prompt,
            max_tokens=50,
            temperature=0.7,
        )
        tecnico_visitante = resposta.choices[0].text.strip()
    else:
        tecnico_visitante = row['tecnico_visitante']

    return pd.Series([tecnico_mandante, tecnico_visitante], index=['tecnico_mandante', 'tecnico_visitante'])

In [ ]:
print(df_filtrado)

## **L**oad

Atualiza o dataframe e preenche o nome de todos os técncios faltantes.

In [ ]:
# Aplique a função 'preencher_tecnicos' para preencher as colunas NaN.
df_filtrado[['tecnico_mandante', 'tecnico_visitante']] = df_filtrado.apply(preencher_tecnicos, axis=1)

# Agora, 'df' contém as colunas de técnicos preenchidas usando a API da OpenAI.